In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob
import json

import pickle
from openai import OpenAI
from pydantic import BaseModel

from tenacity import (
                        retry,
                        stop_after_attempt,
                        wait_random_exponential
)

from tqdm import tqdm

In [2]:
def generate_prompt_predict_possible_joins(target_description, candidate_descriptions, candidate_tables):

    system_msg = f"""
            Given one target column description and many candidate column descriptions, predict all the pairs (candidate table name, candidate 
            description column name) that could be joined.

            Task: Look carefully at the target description of the target column and candidate column descriptions and use this information to identify 
            patterns and relationships between the descriptions, the result must be a list of all the JOINable pairs found. If no joinable pair is 
            found the result should be just the word "none".

            Additional info: A JOIN in relational databases is an operation that retrieves related rows from two tables by linking them 
            based on related  columns between them.
            
            Instructions: 
                1. Look at the target description given to you. 
                2. Look at the candidate descriptions in detail. 
                3. Predict all the highly likely JOINs between these columns based only on these descriptions. Disregard the column names. 
            """
    
    user_msg = f"""Target description:      {target_description}
                   Candidate table:         {candidate_tables}
                   Candidate descriptions:  {candidate_descriptions}
                   Possible JOINs: """.strip()
    
    return system_msg, user_msg

In [3]:
def generate_predictions(target_descriptions, candidate_tables, candidate_descriptions, client):
    
    system_msg_predict_joins, user_msg_predict_joins = generate_prompt_predict_possible_joins(target_descriptions, candidate_tables, candidate_descriptions)
    result = execute_prompt(client, system_msg_predict_joins, user_msg_predict_joins)
    joins = result.choices[0].message.content.split('Possible JOINs: ')[-1].strip()
    
    return joins

In [4]:
@retry(wait=wait_random_exponential(min=1,max=60), stop=stop_after_attempt(6))
def execute_prompt(client, system_msg, user_msg):
    completion = client.chat.completions.create(
                                            model="gpt-4o",
                                            temperature=0.2,
                                            messages=[
                                                        {
                                                            "role": "system", 
                                                             "content": f"{system_msg}"
                                                        },
                                                        {
                                                            "role": "user",
                                                            "content": f"{user_msg}"
                                                        }
                                                    ]
                                            )
    return completion

In [5]:
client = OpenAI()

In [6]:
path = '/home/manoelflorencio/cta_for_jd/testbedXS'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/testbedXS


In [7]:
descriptions = pd.read_csv('Description_test/all_descriptions.csv')
descriptions.head()

,TableName,Column,Description
0,statewise-census-data-in-india-1901-2011.csv,FREQUENCY,The 'FREQUENCY' column consists of the tempora...
1,statewise-census-data-in-india-1901-2011.csv,DATE,The 'DATE' column in the table represents the ...
2,statewise-census-data-in-india-1901-2011.csv,LOCATION_NAME,"The ""LOCATION_NAME"" column in the table repres..."
3,statewise-census-data-in-india-1901-2011.csv,LOCATION_F5,"The column ""LOCATION_F5"" in the table appears ..."
4,statewise-census-data-in-india-1901-2011.csv,LOCATION_ID,"The ""LOCATION_ID"" column contains identifiers ..."


In [8]:
# files = [file.split('/')[-1] for file in glob.glob('datasets/*')]
files = ['eo_pr.csv', 'cultural-spaces.csv', 'public-art.csv', 'libraries.csv', 'schools.csv']
# files = ['cultural-spaces.csv', 'schools.csv']
files

['statewise-census-data-in-india-1901-2011.csv',
 'road-ahead-current-road-closures.csv',
 'property-tie-lines.csv',
 'public-art.csv',
 'gvrd-sewer-trunk-mains.csv',
 'SCS_Staff_Salaries_data_30th_June 2010.csv',
 'schools.csv',
 'rental-standards-current-issues.csv',
 'datasets_579296_1047868_authors.csv',
 'survey_results_schema.csv',
 'animal-control-inventory-lost-and-found.csv',
 'glassdoor_wwfu_val_captions.csv',
 'eo_xx.csv',
 'community-gardens-and-food-trees.csv',
 'road-ahead-upcoming-projects.csv',
 'libraries.csv',
 'cultural-spaces.csv',
 'datasets_517172_952401_train.csv',
 'public-art-artists.csv',
 'eo4.csv',
 'currency_exchange.csv',
 'eo_pr.csv',
 'road-ahead-projects-under-construction.csv',
 'ability_ids.csv',
 'population-by-governorate-citizenship-and-gender.csv',
 'community-centres.csv',
 'street-intersections.csv',
 'population-census-of-botswana-2011.csv']

In [9]:
sample_descriptions = descriptions[descriptions['TableName'].isin(files)]
sample_descriptions = sample_descriptions.reset_index(drop=True)
sample_descriptions.head()

,TableName,Column,Description
0,statewise-census-data-in-india-1901-2011.csv,FREQUENCY,The 'FREQUENCY' column consists of the tempora...
1,statewise-census-data-in-india-1901-2011.csv,DATE,The 'DATE' column in the table represents the ...
2,statewise-census-data-in-india-1901-2011.csv,LOCATION_NAME,"The ""LOCATION_NAME"" column in the table repres..."
3,statewise-census-data-in-india-1901-2011.csv,LOCATION_F5,"The column ""LOCATION_F5"" in the table appears ..."
4,statewise-census-data-in-india-1901-2011.csv,LOCATION_ID,"The ""LOCATION_ID"" column contains identifiers ..."


In [10]:
sample_descriptions['TableName'].value_counts()

TableName
eo4.csv                                                 28
eo_pr.csv                                               28
eo_xx.csv                                               28
public-art.csv                                          19
community-gardens-and-food-trees.csv                    19
cultural-spaces.csv                                     12
SCS_Staff_Salaries_data_30th_June 2010.csv              10
public-art-artists.csv                                   9
statewise-census-data-in-india-1901-2011.csv             9
population-census-of-botswana-2011.csv                   8
rental-standards-current-issues.csv                      8
gvrd-sewer-trunk-mains.csv                               7
animal-control-inventory-lost-and-found.csv              7
datasets_579296_1047868_authors.csv                      6
road-ahead-projects-under-construction.csv               6
road-ahead-upcoming-projects.csv                         6
road-ahead-current-road-closures.csv          

In [11]:
df_dsInformation = pd.read_csv('datasetInformation_testbedXS.csv')

In [12]:
joins_dict = {}

for file in files:
    info  = df_dsInformation[df_dsInformation['filename'] == file]                
    table = pd.read_csv(f'datasets/{file}', delimiter=info['delimiter'].values[0])

    table_descriptions = sample_descriptions[sample_descriptions['TableName'] == file]
    candidate_tables       = sample_descriptions[sample_descriptions['TableName'] != file].iloc[:,0]
    candidate_descriptions = sample_descriptions[sample_descriptions['TableName'] != file].iloc[:,2]

    joins_dict[f'{file}'] = {}
    
    for i in tqdm(range(table_descriptions.shape[0])):
        target_description = table_descriptions.iloc[i, 2]
        joins = generate_predictions(target_description, candidate_tables, candidate_descriptions, client)
        joins_dict[f'{file}'][f'{table_descriptions.iloc[i, 1]}'] = joins

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.33it/s]


In [14]:
with open('Description_test/join_prediction_v2.json', 'w') as file:
    json.dump(joins_dict, file)

In [15]:
with open('Description_test/join_prediction_v2.json', 'r') as file:
    joins_dict = json.load(file)

In [16]:
joins_dict

{'statewise-census-data-in-india-1901-2011.csv': {'FREQUENCY': 'none',
  'DATE': 'none',
  'LOCATION_NAME': '- ("road-ahead-current-road-closures.csv", "LOCATION")\n- ("population-census-of-botswana-2011.csv", "REGION_NAME")',
  'LOCATION_F5': 'none',
  'LOCATION_ID': '1. ("road-ahead-current-road-closures.csv", "LOCATION")',
  'LOCATION_REGIONID': '- (population-census-of-botswana-2011.csv, REGION_REGIONID)',
  'VARIABLE_NAME': 'none',
  'VALUE': 'none',
  'VARIABLE_ID': '- (population-census-of-botswana-2011.csv, REGION_REGIONID)'},
 'road-ahead-current-road-closures.csv': {'PROJECT': 'none',
  'STREET': 'none',
  'LOCATION': 'none',
  'COMP_DATE': '1. (statewise-census-data-in-india-1901-2011.csv, DATE)',
  'URL_LINK': 'none',
  'Geom': 'none'},
 'property-tie-lines.csv': {'Geom': 'none', 'P_ANNOTATION_LBL_ID': 'none'},
 'public-art.csv': {'RegistryID': 'none',
  'ArtistProjectStatement': 'none',
  'Type': 'none',
  'Status': 'none',
  'SiteName': '- ("statewise-census-data-in-india

In [17]:
groundTruth = pd.read_csv('joinable_columns_90containment.csv')
groundTruth

,ds_name,att_name,ds_name_2,att_name_2
0,eo_pr.csv,NAME,eo4.csv,NAME
1,eo_pr.csv,ICO,eo4.csv,ICO
2,eo_pr.csv,STREET,eo4.csv,STREET
3,eo_pr.csv,CITY,eo4.csv,CITY
4,eo_pr.csv,STATE,eo4.csv,STATE
...,...,...,...,...
65,eo4.csv,STATE,eo_xx.csv,STATE
66,eo_xx.csv,ZIP,eo4.csv,ZIP
67,eo_xx.csv,NTEE_CD,eo4.csv,NTEE_CD
68,eo_xx.csv,SORT_NAME,eo4.csv,SORT_NAME


In [18]:
groundTruth[groundTruth['ds_name'].isin(files) & groundTruth['ds_name_2'].isin(files)]

,ds_name,att_name,ds_name_2,att_name_2
0,eo_pr.csv,NAME,eo4.csv,NAME
1,eo_pr.csv,ICO,eo4.csv,ICO
2,eo_pr.csv,STREET,eo4.csv,STREET
3,eo_pr.csv,CITY,eo4.csv,CITY
4,eo_pr.csv,STATE,eo4.csv,STATE
...,...,...,...,...
65,eo4.csv,STATE,eo_xx.csv,STATE
66,eo_xx.csv,ZIP,eo4.csv,ZIP
67,eo_xx.csv,NTEE_CD,eo4.csv,NTEE_CD
68,eo_xx.csv,SORT_NAME,eo4.csv,SORT_NAME


In [19]:
joins_dict

{'statewise-census-data-in-india-1901-2011.csv': {'FREQUENCY': 'none',
  'DATE': 'none',
  'LOCATION_NAME': '- ("road-ahead-current-road-closures.csv", "LOCATION")\n- ("population-census-of-botswana-2011.csv", "REGION_NAME")',
  'LOCATION_F5': 'none',
  'LOCATION_ID': '1. ("road-ahead-current-road-closures.csv", "LOCATION")',
  'LOCATION_REGIONID': '- (population-census-of-botswana-2011.csv, REGION_REGIONID)',
  'VARIABLE_NAME': 'none',
  'VALUE': 'none',
  'VARIABLE_ID': '- (population-census-of-botswana-2011.csv, REGION_REGIONID)'},
 'road-ahead-current-road-closures.csv': {'PROJECT': 'none',
  'STREET': 'none',
  'LOCATION': 'none',
  'COMP_DATE': '1. (statewise-census-data-in-india-1901-2011.csv, DATE)',
  'URL_LINK': 'none',
  'Geom': 'none'},
 'property-tie-lines.csv': {'Geom': 'none', 'P_ANNOTATION_LBL_ID': 'none'},
 'public-art.csv': {'RegistryID': 'none',
  'ArtistProjectStatement': 'none',
  'Type': 'none',
  'Status': 'none',
  'SiteName': '- ("statewise-census-data-in-india

In [26]:
left_table_name = []
left_column_name = []
right_table_name = []
right_column_name = []

for left_table in joins_dict.keys():
    for left_column in joins_dict[left_table].keys():
        if(joins_dict[left_table][left_column] != 'none'):
            for predicted_joins in joins_dict[left_table][left_column].split('\n'):
                try:
                    right_table_name.append(predicted_joins.split('(')[1].split(',')[0])
                    right_column_name.append(predicted_joins.split(')')[0].split(',')[1])
                except:
                    continue
                left_table_name.append(left_table)
                left_column_name.append(left_column)

In [27]:
def remove_extra_quote(string):
    return string.replace('"','').replace("'",'').strip()

In [28]:
d = {'LEFT_TABLE':left_table_name,
     'LEFT_COLUMN':left_column_name,
     'RIGHT_TABLE':list(map(remove_extra_quote,right_table_name)),
     'RIGHT_COLUMN':list(map(remove_extra_quote,right_column_name))}
predicted_joins_df = pd.DataFrame(d)

In [29]:
predicted_joins_df.shape

(84, 4)

In [32]:
pd.set_option('display.max_rows', None)

In [33]:
predicted_joins_df

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN
0,statewise-census-data-in-india-1901-2011.csv,LOCATION_NAME,road-ahead-current-road-closures.csv,LOCATION
1,statewise-census-data-in-india-1901-2011.csv,LOCATION_NAME,population-census-of-botswana-2011.csv,REGION_NAME
2,statewise-census-data-in-india-1901-2011.csv,LOCATION_ID,road-ahead-current-road-closures.csv,LOCATION
3,statewise-census-data-in-india-1901-2011.csv,LOCATION_REGIONID,population-census-of-botswana-2011.csv,REGION_REGIONID
4,statewise-census-data-in-india-1901-2011.csv,VARIABLE_ID,population-census-of-botswana-2011.csv,REGION_REGIONID
5,road-ahead-current-road-closures.csv,COMP_DATE,statewise-census-data-in-india-1901-2011.csv,DATE
6,public-art.csv,SiteName,statewise-census-data-in-india-1901-2011.csv,LOCATION_NAME
7,public-art.csv,SiteName,statewise-census-data-in-india-1901-2011.csv,LOCATION_F5
8,public-art.csv,GeoLocalArea,statewise-census-data-in-india-1901-2011.csv,LOCATION_NAME
9,public-art.csv,GeoLocalArea,statewise-census-data-in-india-1901-2011.csv,LOCATION_F5
